# 📈 Egyszerű makroalapú Forex stratégia
Ez a notebook bemutat egy egyszerű szabályalapú algoritmikus kereskedési stratégiát az EUR/USD devizapárra, makrogazdasági indikátorokat felhasználva (FRED és DXY adatok).

In [ ]:
# Szükséges csomagok telepítése
!pip install fredapi yfinance bt --quiet

In [ ]:
# Könyvtárak importálása
import pandas as pd
import yfinance as yf
from fredapi import Fred
import matplotlib.pyplot as plt

# FRED API kulcs (ezt ingyen regisztrálhatod a https://fred.stlouisfed.org/ címen)
fred = Fred(api_key='ENTER_YOUR_FRED_API_KEY_HERE')

In [ ]:
# Munkanélküliségi ráta lekérése FRED-ből
unemployment = fred.get_series('UNRATE')
unemployment = unemployment.resample('M').last()
unemployment.name = 'Unemployment'

In [ ]:
# DXY árfolyam lekérése a Yahoo Finance-ről
dxy = yf.download('DX-Y.NYB', start='2010-01-01', interval='1mo')['Adj Close']
dxy.name = 'DXY'

In [ ]:
# EUR/USD árfolyam lekérése
eurusd = yf.download('EURUSD=X', start='2010-01-01', interval='1mo')['Adj Close']
eurusd.name = 'EURUSD'

In [ ]:
# Az adatok összefűzése
df = pd.concat([unemployment, dxy, eurusd], axis=1).dropna()

# Egyszerű szabály: ha munkanélküliség > 6% ÉS DXY < 95 → vétel EURUSD
df['Signal'] = (df['Unemployment'] > 6) & (df['DXY'] < 95)
df['Position'] = df['Signal'].shift(1)  # következő hónapban lépünk be

# Hozamok számítása
df['EURUSD_Returns'] = df['EURUSD'].pct_change()
df['Strategy_Returns'] = df['EURUSD_Returns'] * df['Position']

In [ ]:
# Eredmények vizualizálása
(1 + df[['EURUSD_Returns', 'Strategy_Returns']]).cumprod().plot(figsize=(12,6))
plt.title('EUR/USD vs Stratégia teljesítmény')
plt.grid()
plt.show()